In [ ]:
# For use in google colab
#!pip uninstall numba -y
#!pip uninstall librosa -y
#!pip install estival==0.4.8 numpy==1.24.3 nevergrad==0.6.0

In [ ]:
# Required for colab
#import multiprocessing as mp
#mp.set_start_method("forkserver")

In [ ]:
import numpy as np
import pandas as pd

from estival.model import BayesianCompartmentalModel
from estival import priors as esp, targets as est

from estival.sampling import tools as esamp
from estival.sampling.tools import SampleIterator, SampleTypes

from summer2.extras import test_models

In [ ]:
# Begin with a simple test model

m = test_models.sir()

In [ ]:
defp = m.get_default_parameters()
defp

In [ ]:
targetp = defp | {"contact_rate": 0.2}

In [ ]:
m.run(targetp)
m.get_derived_outputs_df()["notifications"].plot()

In [ ]:
m.run(defp | {"contact_rate": 0.2})
targets = [est.NegativeBinomialTarget("notifications", (np.exp(np.random.normal(0.0,0.03, size = (101,))) * m.get_derived_outputs_df()["notifications"]).iloc[::7], 50.0)]

In [ ]:
priors = [esp.UniformPrior("cdr", (0.1,1.0)),
          esp.UniformPrior("contact_rate", (0.01,1.0)),
         ]

In [ ]:
bcm = BayesianCompartmentalModel(m, parameters=defp, targets=targets, priors=priors)

In [ ]:
from estival.sampling import tools as esamp

In [ ]:
from estival.wrappers import nevergrad as eng
import nevergrad as ng

In [ ]:
# Simple nevergrad optimizaton

def optimize_ng(sample):
    opt = eng.optimize_model(bcm, budget=100, opt_class=ng.optimizers.TwoPointsDE, suggested = sample, num_workers=4)
    rec= opt.minimize(100)
    return rec.value[1]

In [ ]:
from estival.utils.parallel import map_parallel

In [ ]:
from estival.utils.sample import SampleTypes

In [ ]:
# Sample 8 points from a Latin Hypercube, then run our nevergrad optimizer over each of them

opt_samples = map_parallel(optimize_ng, bcm.sample.lhs(8, SampleTypes.LIST_OF_DICTS))

In [ ]:
opt_samples

In [ ]:
# Version of the above function that retains indices - we'll see why this is useful soon

def optimize_ng_with_idx(item):
    idx, sample = item
    opt = eng.optimize_model(bcm, budget=100, opt_class=ng.optimizers.TwoPointsDE, suggested = sample, num_workers=4)
    rec= opt.minimize(100)
    return idx, rec.value[1]

In [ ]:
lhs_samples = bcm.sample.lhs(16)

In [ ]:
lhs_samples

In [ ]:
lhs_lle = esamp.likelihood_extras_for_samples(lhs_samples, bcm)

In [ ]:
lhs_sorted = lhs_lle.sort_values("loglikelihood", ascending=False)
lhs_sorted

In [ ]:
best8 = lhs_samples[lhs_sorted.index].iloc[0:8]

In [ ]:
# Sample points inclusive of indices
opt_samples_idx = map_parallel(optimize_ng_with_idx, best8.iterrows())

In [ ]:
opt_samples_idx

In [ ]:
# Get likelihood extras whose indices will be mapped 1:1 with the original pre-optimization samples

lle_samps = esamp.likelihood_extras_for_samples(opt_samples_idx, bcm)
lle_samps

In [ ]:
best_opt_samps = bcm.sample.convert(opt_samples_idx)
best_opt_samps

In [ ]:
init_samps = best_opt_samps.iloc[0:4].convert("list_of_dicts")
init_samps

In [ ]:
import pymc as pm
from estival.wrappers import pymc as epm

In [ ]:
with pm.Model() as model:
    variables = epm.use_model(bcm)
    idata = pm.sample(step=[pm.DEMetropolisZ(variables)],draws=1000, chains=4, initvals=init_samps)

In [ ]:
import arviz as az

In [ ]:
az.summary(idata)

In [ ]:
az.plot_trace(idata, compact=False);

In [ ]:
# Much like like likelihood_extras_for_samples, likelihood_extras_for_idata
# gives detailed likelihoods for the output of pm.sample
# This will only sample the accepted runs and infill the rest of the data
# It can only be used on complete (unburnt, unthinned) idata,
# for anything else, use likelihood_extras_for_samples

lle = esamp.likelihood_extras_for_idata(idata, bcm)

In [ ]:
# Check the per chain performance over time
# We'll use a rolling mean to get a clearer picture

lle["logposterior"].unstack(["chain"]).rolling(100).mean().plot()

In [ ]:
# Get some randomly drawn samples for spaghetti plots
sds = az.extract(idata, num_samples=20)

In [ ]:
spaghetti_res = esamp.model_results_for_samples(sds,bcm)

In [ ]:
spaghetti_res.results["notifications"].plot(legend=False)
bcm.targets["notifications"].data.plot(style='.',color="black")

In [ ]:
# Extract a more comprehensive set of samples for uncertainty

sds = az.extract(idata, num_samples=500)

In [ ]:
# Internally, all estival methods will convert to a SampleIterator;
# it is not necessary to do this manually, but we show it here for clarity

bcm.sample.convert(sds)

In [ ]:
samp_res = esamp.model_results_for_samples(sds,bcm)

In [ ]:
quantiles = esamp.quantiles_for_results(samp_res.results, (0.05,0.25,0.5,0.75,0.95))

In [ ]:
quantiles["notifications"].plot()
bcm.targets["notifications"].data.plot(style='.',color="black")

In [ ]:
# Let's manually construct a SampleIterator to understand its internals a bit more...

def generate_test_si_multiindex(n_chains=3, samples_per_chain=4):
    n_samples = n_chains * samples_per_chain
    index = pd.MultiIndex.from_product([range(n_chains), range(samples_per_chain)], names=["chain", "draw"])  # type: ignore
    components = {
        "a": np.linspace(0.0, 1.0, n_samples),
        "b": np.ones((n_samples, 5)) * 2.0,
        "c": np.ones((n_samples, 10)) * 3.0,
        "d": np.ones(n_samples) * 4.0,
        "e": np.ones(n_samples) * 5.0,
    }
    return SampleIterator(components, index=index)


In [ ]:
sim = generate_test_si_multiindex()

In [ ]:
# It consists of an index...

sim.index

In [ ]:
# ...and some components (columns in a DataFrame, or data_vars in xarray)

sim.components

In [ ]:
# Let's generate a much larger sample

sim = generate_test_si_multiindex(16, 100000)
sim_df = sim.convert("pandas")

In [ ]:
%%time
sim_df.to_hdf("pdsamples_big.h5","samples")

In [ ]:
%%time
sim.to_hdf5("sisamples_big.h5")

In [ ]:
ls

In [ ]:
%%time
sim_loaded = SampleIterator.read_hdf5("sisamples_big.h5")

In [ ]:
%%time
df_loaded = pd.read_hdf("pdsamples_big.h5","samples")

# Sample based methods on BCM

In [ ]:
# SamplePriorsManager is really an independent object that doesn't rely on having a CompartmentalModel

bcm.sample

In [ ]:
spm = bcm.sample

In [ ]:
spm.lhs(4)

In [ ]:
from estival.utils.sample import SampledPriorsManager

In [ ]:
x = esp.GammaPrior.from_mode("x", 2.5, 10.0)
y = esp.UniformPrior("y", [2.0, 4.0])

In [ ]:
x.get_series("pdf").plot()

In [ ]:
y.get_series("pdf").plot()

In [ ]:
pdict = {
    "x": x,
    "y": y
}

In [ ]:
spm = SampledPriorsManager(pdict)

In [ ]:
# Sample generating methods

In [ ]:
from plotly import express as px

In [ ]:
px.scatter_matrix(spm.uniform(64, "pandas"))

In [ ]:
px.scatter_matrix(spm.lhs(64, "pandas"))

In [ ]:
px.scatter_matrix(spm.sobol(64, "pandas"))

In [ ]:
px.scatter_matrix(spm.sobol(64, "pandas", 0.5))

In [ ]:
sobol_samps = spm.sobol(512, "pandas", ci=0.5)

In [ ]:
sobol_samps.hist()

In [ ]:
spm.cdf(sobol_samps).hist()

In [ ]:
spm.ppf(spm.cdf(sobol_samps))